<a href="https://colab.research.google.com/github/alexisboni/ReverseLogisticsOptimization/blob/main/MODELO_DE_LOG%C3%8DSTICA_INVERSA_PARA_EL_MANEJO_DE_RESIDUOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO DE LOGÍSTICA INVERSA PARA EL MANEJO DE RESIDUOS DE VACUNACIÓN COVID EN ESTABLECIMIENTOS DE SALUD UBICADOS EN ZONAS MARGINALES

### LIBRERIAS

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 23.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from pulp import *

### SETS

In [ ]:
# FUNCIÓN PARA PODER HACER SERIES NUMÉRICAS CON TEXTO Y UN NUMERO QUE VARÍE DE 1 EN 1
def serie_str_int(len_range,caracter):
  serie = []
  rang= np.arange(1,len_range+1,1)
  for a in rang:
    serie.append(caracter+ str(a))
  
  return serie

In [ ]:
### SETS DE UNA DIMENSIÓN
# Establecimientos médicos
I= serie_str_int(len_range=32,caracter='EM') # son 32 centros médicos
print(I)

# Centros de Tratamiento
J= ['CT1', 'CT2', 'CT3'] # son 3 centros de tratamiento

# Rellenos Sanitarios
R= ['RS1', 'RS2', 'RS3'] # son 3 rellenos sanitarios

# Periodos de tiempo
T= [1, 2, 3, 4] # son 4 periodos de tiempo


['EM1', 'EM2', 'EM3', 'EM4', 'EM5', 'EM6', 'EM7', 'EM8', 'EM9', 'EM10', 'EM11', 'EM12', 'EM13', 'EM14', 'EM15', 'EM16', 'EM17', 'EM18', 'EM19', 'EM20', 'EM21', 'EM22', 'EM23', 'EM24', 'EM25', 'EM26', 'EM27', 'EM28', 'EM29', 'EM30', 'EM31', 'EM32']


In [ ]:
### SETS DE 2 o 3 DIMENSIONES:
# (Establecimientos médicos,Centros de Tratamiento)
ij = [(CM,CT) for CM in I for CT in J]

# (Centros de Tratamiento,Rellenos Sanitarios)
jr = [(CT,RS) for CT in J for RS in R ]

# (Establecimientos médicos, Periodos)
it = [(CM,t) for CM in I for t in T]

# (Centros de Tratamiento, Periodos)
jt = [(CT,t) for CT in J for t in T ]

# (Rellenos Sanitarios, Periodos)
rt = [(RS,t) for RS in R for t in T ]

# (Establecimientos médicos,Centros de Tratamiento, Periodos)
ijt = [(CM,CT,t) for CM in I for CT in J for t in T ]

# (Centros de Tratamiento,Rellenos Sanitarios, Periodos)
jrt = [(CT,RS,t) for CT in J for RS in R for t in T ]

### PARAMETROS (DATOS)

####LEER EL ARCHIVO CON LOS DATOS

###### El archivo con los datos leídos en el modelo se encuentran en el siguiente link: https://docs.google.com/spreadsheets/d/1FClu1hlgJVLlvvzB-2oiCYUZUfbjmGiSbKf1ag38Azo/edit?usp=sharing

In [ ]:
## Dando acceso a mi google drive para poder leer los datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
!pip install kora
from kora.xattr import get_id

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.6 MB/s 
     |████████████████████████████████| 1.6 MB 41.3 MB/s 


In [ ]:
# Reemplazar por la ubicación donde se guarden los datos en su propio drive
ubicacion='/content/drive/MyDrive/TESIS LOGISTICA INVERSA/DATOS.gsheet'
id_archivo = get_id(ubicacion)
archivo = gc.open_by_key(id_archivo)

In [ ]:
# Leyendo datos de distancia desde cada Establecimiento Médico a cada Centro de Tratamiento:

hoja='dECTij'
worksheet=archivo.worksheet(hoja)

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df_parcial=pd.DataFrame.from_records(rows)
df = df_parcial.iloc[1:,:].copy()
df.columns=['index'] + list(df_parcial.iloc[0,1:])
df.set_index('index', inplace=True)

df = df.replace(',','.', regex=True)
df=df.apply(pd.to_numeric) 

df_dECTij=df.copy()
df_dECTij

,CT1,CT2,CT3
index,,,
EM1,30.14,47.07,16.72
EM2,29.57,46.63,17.39
EM3,28.42,4.54,15.55
EM4,2.92,46.18,16.63
EM5,46.37,61.77,25.75
EM6,27.39,44.60,17.59
EM7,52.23,69.46,38.27
EM8,20.23,37.49,15.02
EM9,24.92,41.53,11.35


In [ ]:
# Leyendo datos de distancia desde cada Centro de Tratamiento a cada Relleno Sanitario:

hoja='dCTRSjr'
worksheet=archivo.worksheet(hoja)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df_parcial=pd.DataFrame.from_records(rows)
df = df_parcial.iloc[1:,:].copy()
df.columns=['index'] + list(df_parcial.iloc[0,1:])
df.set_index('index', inplace=True)

df = df.replace(',','.', regex=True)
df=df.apply(pd.to_numeric) 

df_dCTRSjr=df.copy()
df_dCTRSjr

,RS1,RS2,RS3
index,,,
CT1,34.74,15.11,41.63
CT2,52.09,80.86,58.43
CT3,26.28,29.79,25.48


In [ ]:
# Leyendo datos de Costo de Tratamiento en cada Centro de Tratamiento para cada periodo:

hoja='CTratCTjt'
worksheet=archivo.worksheet(hoja)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df_parcial=pd.DataFrame.from_records(rows)
df = df_parcial.iloc[1:,:].copy()
df.columns=['index'] + list(map(int, list(df_parcial.iloc[0,1:])))
df.set_index('index', inplace=True)

df = df.replace(',','.', regex=True)
df=df.apply(pd.to_numeric) 

df_CTratCTjt=df.copy()
df_CTratCTjt

,1,2,3,4
index,,,,
CT1,1.386,1.386,1.386,1.386
CT2,0.823,0.823,0.823,0.823
CT3,1.800,1.800,1.800,1.800


In [ ]:
# Leyendo datos de Costo de Entierro en cada Relleno Sanitario para cada periodo:

hoja='CEntierroRSrt'
worksheet=archivo.worksheet(hoja)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df_parcial=pd.DataFrame.from_records(rows)
df = df_parcial.iloc[1:,:].copy()
df.columns=['index'] + list(map(int, list(df_parcial.iloc[0,1:])))
df.set_index('index', inplace=True)

df = df.replace(',','.', regex=True)
df=df.apply(pd.to_numeric) 

df_CEntierroRSrt=df.copy()
df_CEntierroRSrt

,1,2,3,4
index,,,,
RS1,0.016,0.016,0.016,0.016
RS2,0.020,0.020,0.020,0.020
RS3,0.021,0.021,0.021,0.021


In [ ]:
# Leyendo datos de Cantidad de Kg de Residuos Covid-19 generados en cada Establecimiento Médico para cada periodo:

hoja='qGeneradait'
worksheet=archivo.worksheet(hoja)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df_parcial=pd.DataFrame.from_records(rows)
df = df_parcial.iloc[1:,:].copy()
df.columns=['index'] + list(map(int, list(df_parcial.iloc[0,1:])))
df.set_index('index', inplace=True)

df = df.replace(',','.', regex=True)
df=df.apply(pd.to_numeric) 

df_qGeneradait=df.copy()
df_qGeneradait

,1,2,3,4
index,,,,
EM1,2.51,2.51,2.51,2.51
EM2,2.95,2.95,2.95,2.95
EM3,2.61,2.60,2.60,2.60
EM4,3.08,3.08,3.07,3.08
EM5,2.32,2.32,2.33,2.32
EM6,2.57,2.56,2.56,2.56
EM7,2.76,2.75,2.79,2.76
EM8,3.65,3.56,3.54,3.58
EM9,3.64,3.61,3.65,3.63


#### CREACION DE LOS DICCIONARIOS CON LOS DATOS

In [ ]:
# FUNCIÓN PARA PODER PASAR LOS DATOS DE DF A DICT
def ParamsDict2D(df, set1, set2):
  d = {}
  for a in set1:
    d[a]={}
    for b in set2:
      d[a][b] = df.loc[a,b]
  
  return d


In [ ]:
#######---- Datos de los Centros de Tratamiento ----#######

# Costo de tratamiento de un Kg de residuo en el CT j en el tiempo t ($/Kg)
CTratCTjt=ParamsDict2D(df_CTratCTjt, J, T)

# Capacidad máxima del CT j (Kg) 
mMaxCTj={'CT1':111000, 'CT2':125000, 'CT3':95000}

# Costo de almacenar un Kg de residuos en el CT j   ($/Kg) 
CAlmacenCTj={'CT1':0.288, 'CT2':0.218, 'CT3':0.309} 

# Costo de almacenar un Kg de residuos  ya tratados en el CT j  ($/Kg)
CTratAlmacenCTj={'CT1':0.180, 'CT2':0.163, 'CT3':0.216} 

# Máxima cantidad de residuos que se pueden almacenar en el CT j (Kg)
mMaxAlmacenCTj={'CT1':49750, 'CT2':55925, 'CT3':42500}

# Máxima cantidad de residuos ya tratados que se pueden almacenar en el CT j (Kg) 
mMaxTratAlmacenCTj={'CT1':50000, 'CT2':57500, 'CT3':45000} 

In [ ]:
#######---- Datos de los Rellenos Sanitarios ----#######

# Costo de entierro de un Kg de residuo en el RS r en el tiempo t ($/Kg)  
CEntierroRSrt=ParamsDict2D(df_CEntierroRSrt, R, T) 

# Capacidad máxima del RS r (Kg) 
mMaxRSr={'RS1':3300000*5, 'RS2':1600000*5, 'RS3': 800000*5} # el dato está por día, se multiplica x5 días a la semana

# Costo de almacenar un Kg de residuos en el RS r ($/Kg) 
CAlmacenRSr={'RS1':0.85/4, 'RS2':0.935/4, 'RS3': 1.0098/4}  # el costo es mensual por eso se divide entre 4

#  Máxima cantidad de residuos que se pueden almacenar en el RS r (Kg)
mMaxAlmacenRSr={'RS1':825000*5, 'RS2':400000*5, 'RS3': 200000*5} 

In [ ]:
#######---- Datos de los Establecimientos de Salud ----#######
# Cantidad de Kg de residuo generado por el establecimiento i en el tiempo t (Kg)
qGeneradait=ParamsDict2D(df_qGeneradait, I, T) 

In [ ]:
#######---- Datos de Transporte ----#######

#  Costo de transporte de un Kg de residuo  en el tiempo t ($/Kg/Km)
CTranst= {1:0.093, 2:0.093, 3: 0.093, 4:0.093}

# Distancia desde el establecimiento i al CT j (Km) 
dECTij= ParamsDict2D(df_dECTij, I, J) 

# Distancia desde el CT j al RS r (Km) 
dCTRSjr=ParamsDict2D(df_dCTRSjr, J, R) 

## DEFINICIÓN DEL MODELO

### VARIABLES

In [ ]:
# Variables continuas
qTransECTijt = LpVariable.dicts("qTransECTijt", ijt, lowBound=0, cat='Continuous')     
qTransCTRSjrt = LpVariable.dicts("qTransCTRSjrt", jrt, lowBound=0, cat='Continuous')   
mTratCTjt = LpVariable.dicts("mTratCTjt", jt, lowBound=0, cat='Continuous')             
mEnterradoRSrt = LpVariable.dicts("mEnterradoRSrt", rt, lowBound=0, cat='Continuous')   
InvCTjt = LpVariable.dicts("InvCTjt", jt, lowBound=0, cat='Continuous')  
InvTratCTjt = LpVariable.dicts("InvTratCTjt", jt, lowBound=0, cat='Continuous') 
InvRSrt = LpVariable.dicts("InvRSrt", rt, lowBound=0, cat='Continuous') 

# Variables Binarias
XusoCTjt = LpVariable.dicts("XusoCTjt", jt, cat='Binary') 
XusoRSrt = LpVariable.dicts("XusoRSrt", rt, cat='Binary') 



### DEFINIR FUNCION OBJETIVO

In [ ]:
model = LpProblem("Modelo de Logística Inversa", LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
model += (lpSum([CTratCTjt[j][t]*mTratCTjt[(j,t)] for j in J for t in T])
+ lpSum([CEntierroRSrt[r][t]*mEnterradoRSrt[(r,t)] for r in R for t in T])
+ lpSum([CTranst[t]*qTransECTijt[(i,j,t)]*dECTij[i][j] for i in I for j in J for t in T])
+ lpSum([CTranst[t]*qTransCTRSjrt[(j,r,t)]*dCTRSjr[j][r] for r in R for j in J for t in T])
+ lpSum([CAlmacenCTj[j]*InvCTjt[(j,t)] for j in J for t in T])
+ lpSum([CTratAlmacenCTj[j]*InvTratCTjt[(j,t)] for j in J for t in T])
+ lpSum([CAlmacenRSr[r]*InvRSrt[(r,t)] for r in R for t in T])
)

### RESTRICCIONES

##### RESTRICCIONES DE EQUILIBRIO EN LA CADENA LOGÍSTICA

In [ ]:
# Todos los residuos generados en los Establecimientos de Salud deben ser enviados a una Planta de Tratamiento
for t in T:
  for i in I:
    model += lpSum([qTransECTijt[(i,j,t)] for j in J]) == qGeneradait[i][t]

In [ ]:
# Todos los residuos que llegan a la Planta de Tratamiento deben ser tratados en algún periodo::
for j in J:
  model += lpSum([qTransECTijt[(i,j,t)] for i in I for t in T]) == lpSum([mTratCTjt[(j,t)] for t in T])

In [ ]:
# Todos los residuos tratados deben ser enviados a un Relleno Sanitario en algún periodo::
for j in J:
  model += lpSum([mTratCTjt[(j,t)] for t in T]) == lpSum([qTransCTRSjrt[(j,r,t)] for r in R for t in T])

In [ ]:
# Todos los residuos que llegan a un Relleno Sanitario deben ser enterrados en algún periodo::
for r in R:
  model += lpSum([qTransCTRSjrt[(j,r,t)] for j in J for t in T]) == lpSum([mEnterradoRSrt[(r,t)] for t in T])

##### RESTRICCIONES DE INVENTARIOS EN LOS ALMACENES

In [ ]:
# Debe haber equilibrio en el inventario de residuos que llegan a los CT, se asume que el inventario inicial es cero; además, al acabar todos los periodos considerados todos los desechos deben ser tratados, esto implica que la cantidad almacenada en el último periodo también sea cero:
for j in J:
  model += InvCTjt[(j,1)] == 0

for t in T[1:]:
  for j in J:
    model += InvCTjt[(j,t-1)] + lpSum([qTransECTijt[(i,j,t)] for i in I]) - mTratCTjt[(j,t)]  == InvCTjt[(j,t)] 

for j in J:
  model += InvCTjt[(j,4)] == 0

In [ ]:
#Debe haber equilibrio en el inventario de residuos ya tratados en los CT, se asume que el inventario inicial es cero; además, al acabar todos los periodos considerados todos los desechos deben haber sido enviados a algún RS, esto implica que la cantidad almacenada en el último periodo también sea cero:
for j in J:
  model += InvTratCTjt[(j,1)] == 0

for t in T[1:]:
  for j in J:
    model += InvTratCTjt[(j,t-1)] + mTratCTjt[(j,t)] - lpSum([qTransCTRSjrt[(j,r,t)] for r in R])  == InvTratCTjt[(j,t)] 

for j in J:
  model += InvTratCTjt[(j,4)] == 0

In [ ]:
# Debe haber equilibrio en el inventario de desechos que llegan para ser enterrados en los RS, se asume que el inventario inicial es cero; además, al acabar todos los periodos considerados todos los desechos deben haber sido enterrados, esto implica que la cantidad almacenada en el último periodo también sea cero:
for r in R:
  model += InvRSrt[(r,1)] == 0

for t in T[1:]:
  for r in R:
    model += InvRSrt[(r,t-1)] + lpSum([qTransCTRSjrt[(j,r,t)] for j in J]) - mEnterradoRSrt[(r,t)]  == InvRSrt[(r,t)]

for r in R:
  model += InvRSrt[(r,4)] == 0

##### RESTRICCIONES DE CAPACIDAD MÁXIMA DE PROCESAMIENTO

In [ ]:
# Los CPT tienen una capacidad de procesamiento limitada; si la planta no es 
# usada, su capacidad y la cantidad tratada debe ser cero: 
for t in T:
  for j in J:
    model += mTratCTjt[(j,t)] <= mMaxCTj[j]*XusoCTjt[(j,t)]

In [ ]:
# Los RS tienen una capacidad de procesamiento limitada; si el RS no es usado, 
# su capacidad y la cantidad tratada debe ser cero:
for t in T:
  for r in R:
    model += mEnterradoRSrt[(r,t)] <= mMaxRSr[r]*XusoRSrt[(r,t)]

In [ ]:
# Si en algún periodo no hay envío de residuos a un CT, 
# este no estaría en uso ese periodo:
for t in T:
  for j in J:
    model += lpSum([qTransECTijt[(i,j,t)] for i in I]) >= XusoCTjt[(j,t)]

# Si en algún periodo no hay envío de residuos a un RS, 
# este no estaría en uso ese periodo:
for t in T:
  for r in R:
    model += lpSum([qTransCTRSjrt[(j,r,t)] for j in J ]) >= XusoRSrt[(r,t)]

##### RESTRICCIONES DE CAPACIDAD MÁXIMA DE ALMACENAMIENTO

In [ ]:
# Los CT tienen una capacidad de almacenamiento limitada 
# de los residuos no tratados
for t in T:
  for j in J:
    model += InvCTjt[(j,t)] <= mMaxAlmacenCTj[j]*XusoCTjt[(j,t)]

In [ ]:
# Los CT tienen una capacidad de almacenamiento limitada 
# de los residuos ya tratados
for t in T:
  for j in J:
    model += InvTratCTjt[(j,t)] <= mMaxTratAlmacenCTj[j]*XusoCTjt[(j,t)]

In [ ]:
# Los RS tienen una capacidad de almacenamiento limitada:
for t in T:
  for r in R:
    model += InvRSrt[(r,t)] <= mMaxAlmacenRSr[r]*XusoRSrt[(r,t)]

### SOLUCION DEL PROBLEMA

In [ ]:
model.solve()

1

In [ ]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [ ]:
### Imprimimos la solución
for v in model.variables():
  if v.varValue !=0:
    print(v.name, "=", v.varValue)

print("El costo mínimo es: ", value(model.objective))

XusoCTjt_('CT1',_1) = 1.0
XusoCTjt_('CT1',_2) = 1.0
XusoCTjt_('CT1',_3) = 1.0
XusoCTjt_('CT1',_4) = 1.0
XusoCTjt_('CT3',_1) = 1.0
XusoCTjt_('CT3',_2) = 1.0
XusoCTjt_('CT3',_3) = 1.0
XusoCTjt_('CT3',_4) = 1.0
XusoRSrt_('RS2',_1) = 1.0
XusoRSrt_('RS2',_2) = 1.0
XusoRSrt_('RS2',_3) = 1.0
XusoRSrt_('RS2',_4) = 1.0
XusoRSrt_('RS3',_1) = 1.0
XusoRSrt_('RS3',_2) = 1.0
XusoRSrt_('RS3',_3) = 1.0
XusoRSrt_('RS3',_4) = 1.0
mEnterradoRSrt_('RS2',_1) = 54.94
mEnterradoRSrt_('RS2',_2) = 54.84
mEnterradoRSrt_('RS2',_3) = 54.89
mEnterradoRSrt_('RS2',_4) = 54.85
mEnterradoRSrt_('RS3',_1) = 41.97
mEnterradoRSrt_('RS3',_2) = 41.92
mEnterradoRSrt_('RS3',_3) = 42.15
mEnterradoRSrt_('RS3',_4) = 42.01
mTratCTjt_('CT1',_1) = 54.94
mTratCTjt_('CT1',_2) = 54.84
mTratCTjt_('CT1',_3) = 54.89
mTratCTjt_('CT1',_4) = 54.85
mTratCTjt_('CT3',_1) = 41.97
mTratCTjt_('CT3',_2) = 41.92
mTratCTjt_('CT3',_3) = 42.15
mTratCTjt_('CT3',_4) = 42.01
qTransCTRSjrt_('CT1',_'RS2',_1) = 54.94
qTransCTRSjrt_('CT1',_'RS2',_2) = 54.84
